##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [1]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 9s 1us/step


In [13]:
y_train[0]

5

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.7281687 , -0.83907455,  0.2764684 ,  0.05559998, -0.39695525,
         0.31323752, -0.22425216, -0.02367566,  0.00819989,  0.8803299 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [14]:
tf.nn.softmax(predictions).numpy()

array([[0.17097011, 0.0356677 , 0.10883015, 0.08726243, 0.05549899,
        0.11290623, 0.06596128, 0.08061174, 0.08322268, 0.1990687 ]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [15]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [16]:
loss_fn(y_train[:1], predictions).numpy()

2.1811976

In [18]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [19]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 135us/sample - loss: 0.2943 - accuracy: 0.9149
Epoch 2/5
60000/60000 [==============================] - ETA: 0s - loss: 0.1433 - accuracy: 0.95 - 7s 123us/sample - loss: 0.1432 - accuracy: 0.9578
Epoch 3/5
60000/60000 [==============================] - 7s 118us/sample - loss: 0.1085 - accuracy: 0.9673
Epoch 4/5
60000/60000 [==============================] - 7s 120us/sample - loss: 0.0879 - accuracy: 0.9729
Epoch 5/5
60000/60000 [==============================] - 7s 120us/sample - loss: 0.0747 - accuracy: 0.9767


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [21]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 1s - loss: 0.0734 - accuracy: 0.9771


[0.0734041824049782, 0.9771]

In [20]:
model.evaluate?

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [22]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [23]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[9.6042811e-07, 4.8041073e-09, 7.1728568e-06, 8.3363066e-06,
        3.2164710e-10, 4.6663320e-08, 4.9351469e-12, 9.9997592e-01,
        5.8680456e-07, 6.9588245e-06],
       [8.0557587e-09, 7.3191128e-04, 9.9911565e-01, 1.4116337e-04,
        6.5332702e-15, 1.1174542e-05, 1.1270785e-08, 1.7538722e-13,
        6.7058664e-08, 4.3186112e-13],
       [5.1241511e-07, 9.9806029e-01, 5.6139441e-05, 3.1028048e-05,
        1.4496010e-04, 4.1141488e-05, 8.0425321e-05, 1.4870940e-03,
        9.3459566e-05, 4.8262918e-06],
       [9.9993980e-01, 4.6084547e-10, 3.0356096e-06, 1.1637085e-08,
        1.4841778e-06, 4.4743484e-07, 4.9010364e-06, 2.6370124e-05,
        9.1030428e-09, 2.4124967e-05],
       [1.5399000e-05, 5.7262310e-09, 5.1973093e-06, 3.8548507e-09,
        9.9928147e-01, 5.4363980e-09, 5.2676256e-07, 1.0189160e-05,
        8.0625142e-07, 6.8638317e-04]], dtype=float32)>